In [ ]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/

In [ ]:
# 查看个人持久化工作区文件
!ls /home/kesci/work/

In [8]:
# 查看当前kernerl下的package

#!pip list --format=columns
!pip install pyecharts

    100% |████████████████████████████████| 163kB 9.1kB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 8.4kB/s ta 0:00:02
  Running setup.py bdist_wheel for simplejson ... done
  Stored in directory: /home/kesci/.cache/pip/wheels/5d/1a/1e/0350bb3df3e74215cd91325344cc86c2c691f5306eb4d22c77
  Running setup.py bdist_wheel for prettytable ... done
  Stored in directory: /home/kesci/.cache/pip/wheels/80/34/1c/3967380d9676d162cb59513bd9dc862d0584e045a162095606
Successfully built simplejson prettytable
You are using pip version 9.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
time: 54.8 s


In [1]:
# 显示cell运行时长
%load_ext klab-autotime

In [10]:
import pandas as pd
df = pd.read_csv('/home/kesci/datas/listings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28452 entries, 0 to 28451
Data columns (total 16 columns):
id                                28452 non-null int64
name                              28451 non-null object
host_id                           28452 non-null int64
host_name                         28452 non-null object
neighbourhood_group               0 non-null float64
neighbourhood                     28452 non-null object
latitude                          28452 non-null float64
longitude                         28452 non-null float64
room_type                         28452 non-null object
price                             28452 non-null int64
minimum_nights                    28452 non-null int64
number_of_reviews                 28452 non-null int64
last_review                       17294 non-null object
reviews_per_month                 17294 non-null float64
calculated_host_listings_count    28452 non-null int64
availability_365                  28452 non-null int64
dt

In [11]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,44054,Modern and Comfortable Living in CBD,192875,East Apartments,NaN,朝阳区 / Chaoyang,39.89503,116.45163,Entire home/apt,792,1,89,2019-03-04,0.85,9,341
1,100213,The Great Wall Box Deluxe Suite A团园长城小院东院套房,527062,Joe,NaN,密云县 / Miyun,40.68434,117.17231,Private room,1201,1,2,2017-10-08,0.10,4,0
2,128496,Heart of Beijing: House with View 2,467520,Cindy,NaN,东城区,39.93213,116.42200,Entire home/apt,389,3,259,2019-02-05,2.70,1,93
3,161902,cozy studio in center of Beijing,707535,Robert,NaN,东城区,39.93357,116.43577,Entire home/apt,376,1,26,2016-12-03,0.28,5,290
4,162144,"nice studio near subway, sleep 4",707535,Robert,NaN,朝阳区 / Chaoyang,39.93668,116.43798,Entire home/apt,537,1,37,2018-08-01,0.40,5,352


time: 86.1 ms


In [12]:
# 看看neighbourhood都有哪些值
df['neighbourhood'].unique()

array(['朝阳区 / Chaoyang', '密云县 / Miyun', '东城区', '西城区', '海淀区',
       '顺义区 / Shunyi', '房山区', '怀柔区 / Huairou', '昌平区', '通州区 / Tongzhou',
       '丰台区 / Fengtai', '大兴区 / Daxing', '延庆县 / Yanqing', '石景山区',
       '门头沟区 / Mentougou', '平谷区 / Pinggu'], dtype=object)

time: 3.44 ms


### 数据预处理

In [13]:
# 由于neighbourhood字段不太规整，所以先将neighbourhood统一格式
def reshape_neighbourhood(x):
    if len(x.split('/'))==1:
        x=x.strip()
    else:
        x=x.split('/')[0].strip()
    return x
df['neighbourhood'] = df['neighbourhood'].apply(reshape_neighbourhood)
df['neighbourhood'].unique()

array(['朝阳区', '密云县', '东城区', '西城区', '海淀区', '顺义区', '房山区', '怀柔区', '昌平区',
       '通州区', '丰台区', '大兴区', '延庆县', '石景山区', '门头沟区', '平谷区'], dtype=object)

time: 115 ms


### 查看各区房源分布

In [14]:
area_counts = df['neighbourhood'].value_counts()
area_counts

朝阳区     10810
东城区      3346
海淀区      3197
丰台区      1758
西城区      1701
通州区      1290
昌平区      1034
密云县       935
顺义区       920
怀柔区       833
大兴区       823
延庆县       718
房山区       579
石景山区      213
门头沟区      152
平谷区       143
Name: neighbourhood, dtype: int64

time: 7.63 ms


*可以看到，房源主要集中在朝阳区*

In [15]:
# 绘制饼图
from pyecharts.charts import Pie
pie = Pie()
pie.add('',zip(area_counts.index.tolist(),area_counts.values.tolist()))
pie.render_notebook()

time: 116 ms


In [16]:
# 绘制地理位置图
from pyecharts import options as opts 
from pyecharts.charts import Map 
from pyecharts.globals import ChartType, SymbolType
c = (
        Map()
        .add(
            "geo",
            [list(z) for z in zip(area_counts.index.tolist(), area_counts.values.tolist())],
            '北京',
        )
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(
            visualmap_opts=opts.VisualMapOpts(max_=11000,is_piecewise=True),
            title_opts=opts.TitleOpts(title="房源分布"),
        )
    )

c.render_notebook()

time: 5.99 ms


### 各区房屋均价

In [17]:
area_group_list = list(df.groupby('neighbourhood')) #分组
area_group_list.sort(key=lambda x:x[1]['price'].mean(),reverse=True) #按照均价排序
area_list,data_list = zip(*area_group_list) #解开成两个序列
price_list = [round(i['price'].mean(),2) for i in data_list]
print(area_list)
print(price_list)

('怀柔区', '延庆县', '昌平区', '平谷区', '密云县', '门头沟区', '东城区', '房山区', '顺义区', '西城区', '大兴区', '朝阳区', '海淀区', '通州区', '石景山区', '丰台区')
[1589.78, 1238.99, 1174.8, 1172.27, 966.96, 751.59, 712.41, 679.15, 667.97, 587.99, 489.54, 484.4, 456.46, 405.55, 403.05, 384.63]
time: 17.1 ms


In [21]:
# 绘制柱状图
from pyecharts.charts import Bar
from pyecharts import options as opts
bar = Bar()
bar.add_xaxis(area_list)
bar.add_yaxis('',price_list)
bar.set_global_opts(
    xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=30)),
    title_opts=opts.TitleOpts(title="各区房屋均价")
    )
bar.render_notebook()

time: 4.29 ms


### 房屋类型

In [22]:
df['room_type'].value_counts()

Entire home/apt    16955
Private room        9838
Shared room         1659
Name: room_type, dtype: int64

time: 7.03 ms


In [23]:
# 绘制饼图
from pyecharts.charts import Pie
pie = Pie()
pie.add('',zip(df['room_type'].value_counts().index.tolist(),df['room_type'].value_counts().values.tolist()))
pie.render_notebook()

time: 35.2 ms


*房屋类型主要以整租为主，合租只占很少一部分*

In [28]:
### 看看每种类型房屋的价格是多少
type_grouped = df.groupby('room_type')
type_grouped['price'].describe().T

room_type,Entire home/apt,Private room,Shared room
count,16955.000000,9838.000000,1659.000000
mean,746.479151,430.681236,299.192887
std,1705.645806,1203.643527,2531.606776
min,0.000000,0.000000,27.000000
25%,356.000000,181.000000,87.000000
50%,470.000000,248.000000,107.000000
75%,658.000000,389.000000,148.000000
max,68983.000000,66667.000000,67909.000000


time: 20.4 ms


*最大值要远远大于75%分位的数据，对整体影响比较大*

In [47]:
(df.query('price>2000').shape[0])/df.shape[0]

0.036728525235484324

time: 83.8 ms


*可以看到价格大于2000的房屋占总数仅仅3%,将这部分去掉再看一下价格的统计*

In [46]:
df.query('price<2000').groupby('room_type')['price'].describe().T

room_type,Entire home/apt,Private room,Shared room
count,16002.000000,9632.000000,1642.000000
mean,529.064804,333.867836,149.293544
std,300.483121,276.240539,146.607866
min,0.000000,0.000000,27.000000
25%,349.000000,181.000000,87.000000
50%,456.000000,242.000000,107.000000
75%,597.000000,369.000000,148.000000
max,1993.000000,1986.000000,1993.000000


time: 64.9 ms


*价格整体上是整租>单间>合租*

### 整体价格分布

In [51]:
df.query('price<2000')['price'].describe()

count    27276.000000
mean       437.272694
std        308.090029
min          0.000000
25%        228.000000
50%        369.000000
75%        537.000000
max       1993.000000
Name: price, dtype: float64

time: 32.4 ms


In [52]:
import matplotlib.pyplot as plt
%matplotlib inline
df.query('price<2000')['price'].plot.hist(bins=20)

<Figure size 432x288 with 1 Axes>

time: 910 ms


*价格集中在100-600之间*

### 看看人气比较高的民宿具有哪些特征

In [53]:
sort_reviews = df.sort_values('number_of_reviews',ascending=False)
sort_reviews[:20]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
325,6622351,梵溪 故宫旁 日式花园庭院 禅意卧房,26047033,Jim,NaN,东城区,39.92917,116.40344,Private room,537,1,322,2019-04-13,6.82,4,339
321,6596814,隐庐 故宫旁 日式花园庭院 新中式LOFT套房,26047033,Jim,NaN,东城区,39.92597,116.39673,Entire home/apt,899,1,284,2019-03-13,6.03,4,345
922,11911698,"东直门,紧邻地铁,温馨聚会空间",24896768,Han,NaN,朝阳区,39.94084,116.44843,Entire home/apt,960,1,264,2019-04-03,7.27,4,352
2,128496,Heart of Beijing: House with View 2,467520,Cindy,NaN,东城区,39.93213,116.42200,Entire home/apt,389,3,259,2019-02-05,2.70,1,93
1254,13506068,"【留凉】雍和宫附近的吃货天堂,居心地のいい部屋,温柔猫咪,全新家具",77464746,Lene,NaN,朝阳区,39.95410,116.42731,Private room,228,1,240,2019-04-08,7.27,1,300
1360,13851394,Art In Nature - Cabin at Sanlitun north,25164545,Chihiro,NaN,朝阳区,39.93950,116.45410,Entire home/apt,409,1,214,2019-04-09,6.39,4,165
877,11539860,胡同里超温暖的小院|近什刹海､南锣鼓巷|壹,42823026,婷婷,NaN,西城区,39.93457,116.36592,Shared room,121,1,212,2019-04-05,5.60,5,74
868,11500895,Cozy & Romantic apartment with ur very own cinema,59658168,Olivia,NaN,朝阳区,39.98521,116.46787,Entire home/apt,376,1,209,2019-04-11,5.54,9,351
2179,15896747,大可的room,25926703,Haidong,NaN,朝阳区,39.99166,116.47757,Shared room,87,1,205,2019-04-12,7.04,8,83
470,7959025,Quiet and newly renovated apartment,16264006,Weakow,NaN,西城区,39.95492,116.36636,Private room,168,1,203,2019-04-12,4.58,3,63


time: 60.1 ms


In [54]:
### 位置
sort_reviews['neighbourhood'][:20].value_counts()

朝阳区    10
东城区     7
西城区     3
Name: neighbourhood, dtype: int64

time: 3.91 ms


*主要集中在朝阳区和东城区*

In [55]:
# 价格
sort_reviews['price'][:20].describe()

count      20.000000
mean      446.350000
std       319.662532
min        87.000000
25%       228.000000
50%       382.500000
75%       579.000000
max      1262.000000
Name: price, dtype: float64

time: 7.97 ms


*价格在400元左右*

### 时间序列可视化分析

In [1]:
# 导入数据
import pandas as pd
df2 = pd.read_csv('/home/kesci/reviews.csv')
# 看看数据类型
df2.info()

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202099 entries, 0 to 202098
Data columns (total 2 columns):
listing_id    202099 non-null int64
date          202099 non-null object
dtypes: int64(1), object(1)
memory usage: 3.1+ MB


In [2]:
# 看看数据是什么样
df2.head()

,listing_id,date
0,44054,2010-08-25
1,44054,2010-10-13
2,44054,2011-08-11
3,44054,2012-04-12
4,44054,2012-08-30


In [3]:
# date是字符串这里要转化为datetime类型方便分析
df2['date'] = pd.to_datetime(df2['date'],format='%Y-%m-%d')

In [4]:
# 增加year,month列
df2['year'] = df2['date'].apply(lambda x:x.year)
df2['month'] = df2['date'].apply(lambda x:x.month)

In [5]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202099 entries, 0 to 202098
Data columns (total 4 columns):
listing_id    202099 non-null int64
date          202099 non-null datetime64[ns]
year          202099 non-null int64
month         202099 non-null int64
dtypes: datetime64[ns](1), int64(3)
memory usage: 6.2 MB


In [6]:
# 以年份来分组
year_grouped = df2.groupby('year')

In [7]:
# 看看每年都有几个月数据
for i in list(year_grouped):
    print('{}:{}'.format(i[0],i[1]['month'].unique().shape[0]))

2010:2
2011:6
2012:12
2013:12
2014:12
2015:12
2016:12
2017:12
2018:12
2019:4


In [20]:
# 看看每年都有多少评论数据
year_grouped.count()

,listing_id,date,month
year,,,
2010,2,2,2
2011,24,24,24
2012,123,123,123
2013,306,306,306
2014,624,624,624
2015,2551,2551,2551
2016,12042,12042,12042
2017,38593,38593,38593
2018,95899,95899,95899


*由于2010,2011,2019数据都没有满12个月，所以不计入下面的统计*

In [16]:
# 选取2012-2018年这七年时间
new_year_grouped = df2.query('year!=2010&year!=2011&year!=2019').groupby('year')

In [18]:
# 看看房租热度随着年份如何变化
import matplotlib.pyplot as plt
%matplotlib inline

new_year_grouped.count()

,listing_id,date,month
year,,,
2012,123,123,123
2013,306,306,306
2014,624,624,624
2015,2551,2551,2551
2016,12042,12042,12042
2017,38593,38593,38593
2018,95899,95899,95899


In [19]:
plt.plot(new_year_grouped.count().index,new_year_grouped.count()['date'])

<Figure size 432x288 with 1 Axes>

*可以看出短租的热度是逐年攀升，特别是在2015年Airbnb正式进入中国市场后开始大幅度增长*

In [21]:
# 看看短租热度和月份有没有关系
month_grouped = df2.query('year!=2010&year!=2011&year!=2019').groupby('month')
month_grouped.count()

,listing_id,date,year
month,,,
1,7064,7064,7064
2,5770,5770,5770
3,4479,4479,4479
4,10016,10016,10016
5,11566,11566,11566
6,12105,12105,12105
7,15317,15317,15317
8,18914,18914,18914
9,16163,16163,16163


In [22]:
plt.plot(month_grouped.count().index,month_grouped.count()['date'])

<Figure size 432x288 with 1 Axes>

*1-3月份是淡季，7-10月份是旺季*

In [36]:
# 单独拿出一年再来验证一下是否符合上面的规律
test_grouped = df2.query('year==2016').groupby('month')
plt.plot(test_grouped.count().index,test_grouped.count()['date'])

<Figure size 432x288 with 1 Axes>

*可以看到两者随月份变化趋势基本相同*

### 评论文本探索

In [2]:
# 解压缩
import zipfile
examplezip = zipfile.ZipFile('/home/kesci/reviews_detail.zip')
examplezip.extractall('/home/kesci/work')
examplezip.close()

In [2]:
# 读取文件
import pandas as pd
df3 = pd.read_csv('/home/kesci/work/reviews_detail.csv')
df3.info()

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202099 entries, 0 to 202098
Data columns (total 6 columns):
listing_id       202099 non-null int64
id               202099 non-null int64
date             202099 non-null object
reviewer_id      202099 non-null int64
reviewer_name    202093 non-null object
comments         201983 non-null object
dtypes: int64(3), object(3)
memory usage: 9.3+ MB


In [3]:
df3.head(),df3.tail()

(   listing_id       id        date  reviewer_id reviewer_name  \
 0       44054    84748  2010-08-25       207019        Jarrod   
 1       44054   118384  2010-10-13       218723      Kimberly   
 2       44054   436978  2011-08-11       609177          Emma   
 3       44054  1118657  2012-04-12      1787536      Andreyna   
 4       44054  2140650  2012-08-30      1179565       Frances   
 
                                             comments  
 0  Sev was very helpful.  Sev showed us where to ...  
 1  We arrived in Beijing very early in the mornin...  
 2  It is a really massive apartment and really co...  
 3  Sev was incredibly helpful, showed us around t...  
 4  The appartment was ideal for our party of 6 ad...  ,
         listing_id         id        date  reviewer_id reviewer_name  \
 202094    33889408  438178936  2019-04-16    255996319            小倩   
 202095    33890728  438182720  2019-04-16    255995373            志辉   
 202096    33891613  438182693  2019-04-16    

- 评论里面有英文还有中文
- 下面想将中英文分离开来
- 分别对中英文进行词云展示
- 对中文使用snownlp做简单的情感倾向分析

In [4]:
# 编写函数分离中英文
def get_chinese(x):
    for i in str(x):
        if ('\u4e00' <= i <= '\u9fa5'):
            return True
        else:
            return False
df3['test'] = df3['comments'].apply(get_chinese)

*分离中英文这个过程当然也可以放到遍历comments写入文件时操作，但是通过for循环来处理这20多万条文本还是比较慢的*

In [5]:
# 看一下分离的效果
# 随机抽取5条中文
df3['comments'][df3['test']].sample(5)

26801                    可以了解一下如何更方便找到梯口｡可以了解附近是否有自己需要的消费场景
116421    房子的地理位置很好,走路去奥体公园,鸟巢水立方很近,出小区门就有公交站台,一切都还蛮方便,设...
132355                                      房间很大很干净,适合大家庭入住
83686     房间虽然不大,但是很干净舒适. 房间的位置也非常好,在胡同里的一个小区,地铁站就在附近,周围...
143442    离地铁站近  交通便利   购物和逛街都非常方便   室内装修和环境都非常棒!设施非常齐全 ...
Name: comments, dtype: object

In [6]:
# 随机抽取5条英文
df3['comments'][~df3['test']].sample(5)

133759    Vv is super nice and always responds to messag...
30271     A beautiful little hi-tech home in a tradition...
86326     The accommodation is conveniently situated in ...
20098     Vincent was a very nice and accommodating host...
44293     The sofa is very comfortable, but don't expect...
Name: comments, dtype: object

*将中英文分别写入文件*

In [7]:
def write_file(name,comments):
    f = open('/home/kesci/work/{}.txt'.format(name),'ab')
    for i in comments:
        f.write(str(i).encode('utf-8'))
        f.write(b'\n')
    f.close()

In [8]:
# 写入中文
write_file('chinese_comments',df3['comments'][df3['test']])

In [9]:
# 写入英文
write_file('english_comments',df3['comments'][~df3['test']])

In [13]:
# 分词绘制中文词云
from collections import Counter
import jieba
import json

f = open('/home/kesci/work/chinese_comments.txt',"r",encoding='utf-8').read()
# 分词
words = jieba.lcut(f)
# 统计词频
word_counts = dict(Counter(words))
# 导入停词
with open('/home/kesci/work/chinese_stop_words.json','r') as file:
    stop_words = json.load(file)
index,excludes = zip(*stop_words.items())
# 删除停用词
for word in excludes:
    del word_counts[word]
items = list(word_counts.items())
# 对字典排序
items.sort(key=lambda x:x[1],reverse=True)
items[:40]

[('好', 193358),
 ('房东', 159954),
 ('房间', 126240),
 ('方便', 100928),
 ('干净', 84978),
 ('房子', 83068),
 ('不错', 57804),
 ('入住', 57324),
 ('位置', 56610),
 ('特别', 53936),
 ('我们', 53276),
 ('可以', 50018),
 ('北京', 49400),
 ('体验', 47356),
 ('热情', 45422),
 ('交通', 38066),
 ('舒服', 37982),
 ('地铁站', 37922),
 ('超级', 32548),
 ('不', 31306),
 ('地理位置', 31098),
 ('下次', 31022),
 ('一个', 29824),
 ('温馨', 29612),
 ('推荐', 29100),
 ('便利', 28562),
 ('小区', 28236),
 ('整洁', 27780),
 ('设施', 27702),
 ('喜欢', 27566),
 ('近', 27536),
 ('齐全', 26236),
 ('很棒', 25794),
 ('性价比', 25710),
 ('装修', 24766),
 ('舒适', 24628),
 ('地铁', 24106),
 ('一次', 23424),
 ('安静', 22904),
 ('附近', 22720)]

In [16]:
# 绘制词云
# !pip install pyecharts
from pyecharts import options as opts
from pyecharts.charts import Page, WordCloud
from pyecharts.globals import SymbolType
word = WordCloud()
word.add('',items[:40],word_size_range=[20, 100],shape=SymbolType.DIAMOND)
word.set_global_opts(title_opts=opts.TitleOpts(title="comments WordCloud"))
word.render_notebook()

1. '好','方便','喜欢','不错','推荐'可以看出大部分租客还是比较满意的
2. 租客关注点偏向一下四项
	- 房主：'房东','热情'
	- 房子：'房间','设施','干净','整洁','装修'
	- 位置：'地理位置','附近','地铁站','交通'
	- 价格：'性价比'

In [19]:
# 看看提到'房东'的都说些什么
df3['comments'][df3['test']][df3['comments'][df3['test']].str.contains('房东')].sample(10)

23055               和房东沟通很及时,有问题都会及时解答｡房间卫生清洁的不是很好,不过整体上很不错
176017                房东挺好的,深夜订的,打扰他睡觉了,房子有点不好找,可能是我对北京不熟吧｡
124805    正值中秋,朋友想住北京四合院所以我们有机会选择了Amour的私家小院,整体干净有格调,房东人...
170997    房东人很好沟通~而且地理位置很好,就在地铁旁出行很方便｡而且一定要看房东写的的出行指南,直接...
162369                                           房东的装修品味很不错
178262                              房东人很好,房间很干净很温暖,愉快的旅行体验!
23514     豆汁儿和宝多多超可爱,前几天遇到的其他房客也都很热情大家有交流也一起去吃了火锅,就很开心,房...
157982    迄今为止最满意的一次入住体验,fei ch非常感谢,房东很热情,you we有问必答,房间干...
25657             房东很热情  还给了附近吃的攻略 房间很漂亮很干净 希望下次还有机会呢入住您的丸子
137047    房东人很好,房子环境很好,有家的感觉,东西设备齐全,环境很安静,喜欢房间里的小地毯~出门就可...
Name: comments, dtype: object

*基本上都是对房东的赞赏*

### NLP情感倾向统计